![STSCI banner](https://github.com/STScI-MIRI/MRS-ExampleNB/raw/main/assets/banner1.png)

# Introduction to TSO data products & the JWST TSO pipeline

Author: Sarah Kendrew, Instrument & Calibration Scientist, ESA/STScI MIRI Branch <br>
Last Updated: 17 Nov 2021<br>
Pipeline version: 1.3.3<br>

### Table of contents

1. [Introduction](#intro)<br>
   1.1 [Purpose of this Notebook](#purpose)<br>
   1.2 [Input Simulations](#inputs)<br>
   1.3 [Caveats for Simulated Data](#mirisim)<br>





# 1.<font color='white'>-</font>Introduction <a class="anchor" id="intro"></a>

## 1.1<font color='white'>-</font>Purpose of this notebook <a class="anchor" id="purpose"></a>

In this notebook we provide a realistic example for running the JWST pipeline on a JWST Time Series Observation (TSO). For the purposes of this tutorial we will work with a simulated observation with the MIRI Low Resolution Spectrometer (LRS). In particular, we focus on aspects of the pipeline that differ from "standard" algorihtms and procedures. There will not be enough time to look at every step in detail, but we will demonstrate how to make changes to the pipeline setting (and why you might want to do that) for the best scientific utility. 

Note that the notebook uses JWST Calibration Pipeline version 1.3.3, which is the current version at the time of this JWebbinar. The pipeline will however be further developed and updated post-launch. 

We will start with a simple simulated MIRI LRS observation, created using MIRISim version 2.4.1, which is compatible with pipeline version 1.3.3 (https://wiki.miricle.org/Public/MIRISim_Public). The data are described in more detail below. 


## 1.2<font color='white'>-</font>Input Simulations <a class="anchor" id="inputs"></a>

We used the MIRISim software package (v2.4.1) to generate realistic simulations of a MIRI LRS slitless observation of a simple stellar-type point source. The stellar SED was modelled as a simple black body spectrum with the following parameters:

* Temperature = 6230 K
* Normalised to K = 8.99, or flux of 20 mJy at 2 $\mu$m. 

LRS slitless observations are carried out in the SLITLESSPRISM subarray of the MIRI Imaging detector. The subarray has 416 rows x 72 columns (the left-most 4 columns are reference pixels, i.e. no illuminated), with sampling of 0.11 arcsec/pix. The single-frame read time for this subarray is 0.159 seconds, and the FASTR1 read mode has an extra reset between integrations. We perform an observation of 100 groups, 10 integrations, in a single exposure; giving an exposure time of:

t$_{exp}$ = ((100 + 1) $\times$ 0.159) $\times$ 10 = 160.59 s = 2.67 minutes

"Real" TSOs will typically have many more integrations than 10, with exposures covering many hours. But for the sake of reducing processing time, we use a shorter exposure here. 



### 1.3<font color='white'>-</font>Caveats for Simulated Data<a class="anchor" id="mirisim"></a> ###

As noted above, in this notebook we will be processing simulated data created with the MIRIsim tool.  Like the pipeline, MIRISim is also an evolving piece of software and there are multiple known issues that can cause problems.  

**General MIRISim caveats**

- Detector noise properties are not modelled in a fully physically realistic way. It's not recommended to use these simulations "out of the box" for detailed noise investigations. 

- Reference pixels are not treated consistently, the refpix step of detector1 must therefore be turned off to process mirisim data without artifacts.

- The default detector read mode is FASTR1, which contains a reset between integrations. MIRISim still uses the FAST read mode, which does not include this extra reset. The exposure time recordedin the simulated FITS header is therefore missing this additional reset time. 

**MIRI TSO-specific caveats**

- There is no "TSO" flag in MIRISim; the software does not set the header keyword that the pipeline looks for to recignise whether an exposure is a TSO. We will set this manually in this notebook.

- MIRISim is not able to insert a time-variable signal into a simulated observation. The source flux is assumed to be constant. Additional tools have been developed in the MIRI consortium for this purpose, but for the aims of this tutorial we do not consider this. 

- The MIRI LRS prism has a leak in its transmission that causes some 3-4 $\mu$m flux to contaminate the spectrum around 6-7 $\mu$m. This is not included in the MIRISim models. 




<hr style="border:2px solid gray"> </hr>

## 2.<font color='white'>-</font>Setup <a class="anchor" id="setup"></a>

In this section we set things up a number of necessary things in order for the pipeline to run successfully:

1. import the necessary python packages
2. specify the directory structure

First the imports.


In [ ]:
# Need to set these enviromental variables for this notebook to work properly:
%set_env CRDS_PATH $HOME/crds_cache
%set_env CRDS_SERVER_URL https://jwst-crds.stsci.edu


import os
import glob
import shutil

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import astropy.io.fits as fits

from jwst import datamodels
from jwst.pipeline import calwebb_detector1, calwebb_spec2, calwebb_tso3
from jwst.associations.asn_from_list import asn_from_list
from gwcs.wcstools import grid_from_bounding_box

Next we will create an output directory for the new products we produce. 

In [ ]:
outdir = 'miri_lrs_output'
if not os.path.exists(outdir):
    os.mkdir(outdir)


<hr style="border:1px solid gray"> </hr>

## 3.<font color='white'>-</font>Retrieving and Inspecting the Uncalibrated Data <a class="anchor" id="firstlook"></a>

In practice, you will retrieve your observational data from the [MAST archive portal](https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html). Here you can query on a host a different parameters - mission, instrument, program ID, date of execution, etc. 

In the archive you will find both uncalibrated and calibrated data. All data received from the observatory is automatically processed by the current pipeline version; this includes ancillary data such as target acquisition images. The steps that are run in the pipeline for a particular data mode can be found in the pipeline documentation for [Stage 1](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_detector1.html#calwebb-detector1), [Stage 2](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_spec2.html) and [Stage 3](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_tso3.html). 

This automated pipeline processing uses a set of default parameters that were determined by the instrument teams to be "good" for a typical observation, however, some observers may want to try out different settings or find better settings for their particular science case. We will show later how this works. 

For the purposes of this JWebbinar we provide a simulated observation. 


In [ ]:
f = 'miri_lrs_tso_100G10I_mirisim241.fits'
#f = '/home/shared/preloaded-fits/miri_lrs_tso_100G10I_mirisim241.fits'

You can access this file and examine structure & contents using regular astropy FITS tools, like this. You can see the file has 5 extensions:

* An empty primary extension (with a header)
* SCI: the science data. This is a 4D dataset: NCOLS x NROWS x NGROUPS x NINTS. 
* PIXELDQ: data quality flags. This is a single 2D plane with dimensions NCOLS x NROWS.
* REFOUT: the MIRI reference output values. This is a MIRI specific output. 
* ASDF: the metadata for the datamodels. 

Note that normally the PXELDQ plane is added in the first pipeline stage; its presence here is a feature of MIRISIm. Only the reference columns are flagged, the rest of this array is zeros. 

_Note to Sarah: what is the pixel DQ flag 16777217 in that 5th column of the DQ array?_

In [ ]:
hdu = fits.open(f)
hdu.info()

Using matplotlib, we can visualize the data. The first plot below shows groups 25, 50 and 100 of the first integration of the exposure. You can see how the signal builds up with increasing group number. This increase in signal forms the ''ramp'' that the pipeline will be fitting. 

The second plot shows this flux build up in 2 pixels: one in the source spectrum, and one in the background. 

_Note SK: make this figure look prettier_

In [ ]:
uncal_fits = hdu[1].data
print(np.shape(uncal_fits))

nints = np.shape(uncal_fits)[0]

# identifying pixels in source and background regions
src_px = [37, 360]
bgr_px = [20, 200]
px_labels = ["source + bgr px", "bgr px"]


ql_fig, ax = plt.subplots(ncols=3, nrows=1, figsize=[15,8])
plt_ints = [24, 49, 99]



for i, pi in enumerate(plt_ints):
    ax[i].imshow(uncal_fits[8, pi, :, :], origin='lower', aspect='equal', interpolation='None')
    ax[i].set_title('Grp {}'.format(pi+1))
    sc1 = ax[i].scatter(src_px[0], src_px[1], marker='x', color='cyan')
    sc2 = ax[i].scatter(bgr_px[0], bgr_px[1], marker='x', color='magenta')

ql_fig.legend([sc1, sc2], labels=px_labels, loc='lower center', fontsize='x-large', ncol=2)




In [ ]:


ramp_fig, ax = plt.subplots()
for i in range(nints):
    ax.plot(uncal_fits[i,:,src_px[1], src_px[0]], 'c-', lw=2)
    ax.plot(uncal_fits[i,:,bgr_px[1], bgr_px[0]], 'm-', lw=2)

leg = [Line2D([0], [0], lw=2, color='c', label='source + bgr px'), 
       Line2D([0], [0], lw=2, color='m', label='bgr px')]   
    
ax.set_xlabel('group no.', fontsize='large')
ax.set_ylabel('DN', fontsize='large')
ax.legend(handles=leg, loc=2, fontsize='large')

ramp_fig.show()


### 3.1 Introduction to JWST datamodels

The JWST Calibration Pipeline ("the pipeline") provides datamodels for convenient accessing and working with the data. These datamodels are effectively containers that are optimised for particular JWST data types. More information is available [here](https://jwst-pipeline.readthedocs.io/en/latest/jwst/datamodels/index.html).

You don't need to know what specific model your data corresponds to. The ``datamodels.open`` function checks the relevant header keywords and matches the data against an existing model. 

In the cells below we will explore come useful aspects of the datamodels. 

In [ ]:
uncal = datamodels.open(f)
print(uncal)
print(uncal.info())


This schema gives us a first look at the the model attributes, and how the metadata is packaged in the model. Most importantly, the science data is in the ``uncal.data`` attribute.  If you aren't sure where to find a particular keyword, the function ``uncal.find_fits_keyword()`` can identify it for you. 

In [ ]:
print('The number of groups per integration in the exposure is {0}'.format(uncal.ngroups))
print('The number of integrations in the exposure is {0}'.format(uncal.nints))
print('Information on the filter used in this exposure can be found here: {0}'.format(uncal.find_fits_keyword('FILTER')))
print('OK! So the filter used is {0}, which is the LRS double prism')



We can recreate the above plot using datamodel syntax

In [ ]:
ql_fig2, ax2 = plt.subplots(ncols=3, nrows=1, figsize=[15,8])
plt_ints = [24, 49, 99]

for i, pi in enumerate(plt_ints):
    ax2[i].imshow(uncal.data[8, pi, :, :], origin='lower', aspect='equal', interpolation='None')
    ax2[i].set_title('Grp {}'.format(pi+1))
    sc1 = ax2[i].scatter(src_px[0], src_px[1], marker='x', color='cyan')
    sc2 = ax2[i].scatter(bgr_px[0], bgr_px[1], marker='x', color='magenta')

ql_fig2.legend([sc1, sc2], labels=px_labels, loc='lower center', fontsize='x-large', ncol=2)





The ``meta`` attribute of the model is particular rich in unseful information. We'll show some of its useful features in the next cell. Not all attributes of the model are correctly populated as the MIRISim simulation data lacks some of the JWST observatory keywords. 


We show here that the "TSOVISIT" keyword or attribute is not set. This is a MIRISim issue, and we'll show below how we set this manually in the data model to ensure that the pipeline recognizes the exposure as a TSO.

In [ ]:
print('Total exposure time is {:.2f} seconds'.format(uncal.meta.exposure.exposure_time))
print('Detector readout pattern was {0} mode'.format(uncal.meta.exposure.readpatt))      
print('The original filename of this RampModel was {0}'.format(uncal.meta.filename))     
print('The most amazing space telescope bar none is {0}'.format(uncal.meta.telescope))   
print('TSOs should have this attribute set to True and instead it reads {0}'.format(uncal.meta.visit.tsovisit))

# Let's change the TSO status
if not uncal.meta.visit.tsovisit:
    uncal.meta.visit.tsovisit = True

print('Now the TSO setting reads {0}'.format(uncal.meta.visit.tsovisit))


### Summary and further reading

In this section, we:
* loaded the uncalibrated MIRISim data for a slitless LRS exposure
* showed how to load the data using datamodels
* showed some of the useful features of datamodels and how metadata is organised in the model attributes
    
    
Further reading
*




## 4.<font color='white'>-</font>Running the Pipeline <a class="anchor" id="pipeline"></a>

In this section we will demonstrate how to run the JWST calibration pipeline on TSO data. When you retrieve your data from the archive, you will have pipeline processed data available for download already; however as part of your research you may well want to rerun the pipeline, or specific steps or stages, to optimise certain settings for your targets.

We will highlight here a few pipeline steps that you may want to skip or modify for TSOs, and show how to do that. 

The pipeline consists of [**3 stages**](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/main.html#pipelines). For TSOs, these are:

* CalDetector1: implements basic detector calibrations and converting ramps to slopes
* CalImage2 or CalSpec2: performs additional instrument- or mode-level calibrations, returning flux calibrated images for imaging data, and flux calibrated images and extracted spectra for spectroscopic data. 
* CalTso3: higher level calibrations, returning more TSO-specific output products

In the next sections we will run our MIRI LRS data through these pipeline stages, highlighting basic functionality, output products and suggested modifications. We will start with an introduction to the basic call sequence for the pipeline. 



### 4.1 Intro to running the pipeline

Full documentation for running the pipeline from within Python can be found on the [main pipeline documentation pages](https://jwst-pipeline.readthedocs.io/en/latest/jwst/introduction.html#running-from-within-python). In additional, materials for other JWebbinars providing more general instroductions to the pipeline can be found [here](https://www.stsci.edu/jwst/science-execution/jwebbinars).

In our import statements above we imported the modules ``calwebb_detector1``, ``calwebb_spec2`` and ``calwebb_tso3`` from ``jwst.pipeline``. The most basic call sequence looks as follows:

``det1 = calwebb_detector1.Detector1Pipeline.call(f)`` passing the FITS file to the call sequence; or<br>

``det1 = calwebb_detector1.Detector1Pipeline.call(uncal)`` using the previously loaded datamodel as input. <br>

In this case we will work with the ``uncal`` datamodel, as we made a modification to it in the above section. Let's see what happens. 




In [ ]:
#print(uncal.meta.visit.tsovisit)
det1 = calwebb_detector1.Detector1Pipeline.call(uncal, output_dir=outdir, save_results=True)
print(det1)
!ls miri_lrs_output


If we see no errors, the pipeline successfully ran on our data. The print statement for ``det1`` shows that the pipeline has produced an ImagModel with dimensions (416, 72), which are the correct dimensions for our subarray, corresponding to the output file miri_lrs_tso_100G10I_mirisim241_rate.fits. This is the slope image in DN/s, co-added over all integrations. 

We can see that the output directory contains an additional file, miri_lrs_tso_100G10I_mirisim241_rateints.fits. This is the file that contains the slope images for each integration, which is the product we want to use for TSOs. Let's take a look at the contents of this file. 

In [ ]:
rifile = glob.glob(outdir+'/*rateints.fits')
print(rifile)

rate_ints = datamodels.open(rifile[0])
print(rate_ints)

sl_fig, axs = plt.subplots(ncols=5, nrows=2, figsize=[15,10])

for i,aa in enumerate(axs.flat):
    im = aa.imshow(rate_ints.data[i,:,:], origin='lower', aspect='equal', interpolation='None')
    aa.set_title('int = {0}'.format(i+1))

    
sl_fig.subplots_adjust(right=0.9)
cbar_ax = sl_fig.add_axes([0.95, 0.2, 0.02, 0.6])
cbar = sl_fig.colorbar(im, cax=cbar_ax)
cbar.set_label('DN/s', fontsize='x-large')
sl_fig.show()


OK, let's now dig a bit deeper in what the pipeline is doing to learn more about reference files and to prepare for making some changes. The steps the pipeline runs for a particular mode are defined in a [parameter file in ASDF format](https://jwst-pipeline.readthedocs.io/en/latest/jwst/stpipe/config_asdf.html#config-asdf-files) (ASDF stands for Advanced Scientific Data Format). Asdf file can be read with a simple text editor, and a python package ``asdf`` exists to access programmatically.  

There's a handy function called ``get_crds_parameters`` that prints out all the metadata in the model, including information on the provenance of the file, data units, the calibration steps run (the ``cal_step`` entries), the version of the pipeline that was used (``calibration_software_version``), the names of the reference files used for calibration (``ref_file`` entries) and more.




In [ ]:
rate_ints.get_crds_parameters()

### 4.2 Retrieving and examining reference files

Reference files and parameter files for the calibration pipeline live in the **JWST Calibration Reference Data System** - [CRDS](https://jwst-pipeline.readthedocs.io/en/latest/jwst/introduction.html#crds). You can see this acronym referenced throughout the pipeline documentation, and in the metadata for the exposures. 

Each version of the pipeline has an associated CRDS **context**; you can see the context used in our Detector1Pipeline run in ``rate_ints.meta.ref_file.crds.context_used`` as ``jwst_0776.pmap``. Sometimes it may be necessary to change this setting, in order to run the pipeline with older reference files. 

The CRDS system can be accessed programmatically, or simply via the following URL: https://jwst-crds.stsci.edu, from where you can download reference files with a simple click. In the figures below you can see the web interface for CRDS, and how we can locate the parameter ASDF files for individual pipeline stages and steps. See if you can retrieve the ASDF file used for our Detector1Pipeline run?

In the cell below we then demonstrate how to access a CRDS file programmatically and override it in a pipeline run.

![Fig 1 CRDS interface](images/crds_interface.png)

![Fig 2 Det1 parameter file](images/crds_miri_det1.png)

In [ ]:
from astropy.utils.data import download_file


read_noise_file = (rate_ints.meta.ref_file.readnoise.name).split('/')[-1]
print(read_noise_file)
file_path = download_file('https://jwst-crds.stsci.edu/unchecked_get/references/jwst/'+read_noise_file)
os.rename(file_path, read_noise_file)

#reference files can also be read in with JWST datamodels!
read_noise = datamodels.open(read_noise_file)
print(read_noise)

rd_fig, rd_ax = plt.subplots(figsize=[3,9])
rd_im = rd_ax.imshow(read_noise.data, origin='lower', aspect='equal', interpolation='None')
rd_fig.colorbar(rd_im, ax=rd_ax)
rd_fig.show()

OK, we've now seen how to run the pipeline with its default settings, and where to find the information about those settings. Via the datamodel metadata we can see the filenames of any reference files that were used, the CRDS context. We can access these files from a web interface or programmatically. 

Next, let's see how we can make changes to these settings. 

<hr style="border:2px solid gray"> </hr>

### 4.3 Making changes to the pipeline steps

It will be a pretty common occurrence that you want to investigate the impact of a particular calibration step on your analysis, change a parameter, or try using a different reference file. In this section we'll look at how to do that. 

__First__, we will take the above read noise reference file, change its filename and rerun the ``Detector1Pipeline()`` step with the new (but unchanged) file. This demonstrates how a reference file can be _overridden_. The read noise reference file is references in both the jump step and in the ramp fitting. 

(**Note:** The basic detector calibration reference files were created following very detailed ground testing and often lengthy detector investigations. We show here how to override the read noise file to demonstrate the _method_; we recommend you replace Detector1 reference files with _extreme caution_). 

__Second__, we are finding that the jump step is flagging too many pixels as cosmic rays. We want to increase the jump threhold where the step flags the pixel as a cosmic. Here we demonstrate how to change a step _parameter_. 

__Third__, we would like the ``Detector1Pipeline()`` to run the ``lastframe`` step. The MIRI detectors have an odd pull-down effect on the last frame (group) in an integration. This introduces a deviation from the (quasi-) linear response of the detector, and as the effect has an odd-even row dependence, it can impact line measurements, especially if we are looking at line ratios. However, this effect is also stable over time for a given source, so for TSOs we prefer to use the last frame for maximal SNR. But if we consider a situation where the absolute flux calibration _is_ important - for example to match the data with that from another instrument, then we may want to rerun the pipeline to include the step (and _exclude_ the last frame in the ramp fit). 





In [ ]:
# Change 1: copy the read noise reference file to a different file (make no other changes)
new_readnoise = 'miri_new_readnoise_file.fits'
shutil.copyfile(read_noise_file, new_readnoise)

# Change 2: set a new rejection threshold
rej_thresh = 10.

# Bring it all together
d1mod = calwebb_detector1.Detector1Pipeline.call(uncal, output_dir=outdir, save_results=True,
                                           steps={'jump': {'rejection_threshold':rej_thresh, 'override_readnoise': new_readnoise},
                                                  'ramp_fit': {'override_readnoise': new_readnoise},
                                                  'lastframe': {'skip': False}})



In [ ]:
# Now let's check if our changes were applied
print('Read noise reference file used was {0}'.format(d1mod.meta.ref_file.readnoise.name))
print('The last frame step status: {0}'.format(d1mod.meta.cal_step.lastframe))
d1mod.